## ewf-ext-02-03-07 - NDVI anomalies of growing season per parcel 

NDVI anomalies of growing season per parcel

---

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'NDVI anomalies of growing season per parcel'),
                ('abstract', 'NDVI anomalies of growing season per parcel'),
                ('id', 'ewf-ext-02-03-07')])

### <a name="parameter">Parameter Definition 

In [ ]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON ((-8.864205 38.88616500000001, -8.864205 38.986165, -8.964205000000002 38.986165, -8.964205000000002 38.88616500000001, -8.864205 38.88616500000001))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

In [ ]:
nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value', 'P001'),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

In [ ]:
indexAndApiKeys = dict([('id', 'indexAndApiKeys'),
                        ('value', ''),
                        ('title', 'index,apikey pairs'),
                        ('abstract', 'index,apikey pairs'),
                        ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the NDVI stats' identifiers

In [ ]:
input_identifiers = ('97AE147C8995EA5FDA87B6342C3E6C20C41172D8', 'D0D7F08C8BFAEC14C11ACA6C824AB487764A48F8')

**Input references**

This is the MODIS stack catalogue references

In [ ]:
input_references = ('https://catalog.terradue.com/better-ext-02-03-02/search?format=atom&uid=97AE147C8995EA5FDA87B6342C3E6C20C41172D8','https://catalog.terradue.com/better-ext-02-03-06/search?format=atom&uid=D0D7F08C8BFAEC14C11ACA6C824AB487764A48F8')

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = ""

**Aux folders**

In [ ]:
output_folder = ''

#### Import Modules

In [ ]:
import os
import shutil

import sys
import string
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.wkt import loads

import datetime


#import pdb

import pandas as pd
import geopandas as gpd

import copy

import cioppy
ciop = cioppy.Cioppy()

#### Auxiliary vars

In [ ]:
check_results = True

In [ ]:
# get metadata from catalog
def get_input_metadata (input_refs, apikeys):
    
    # for each product get metadata
    Result_Prod = []
    
    for index,product_ref in enumerate(input_refs):
        
        for index in apikeys:
            if index in product_ref:
                cat_index = index
                cat_apikey = apikeys[index]
        
        # since the search is by identifier
        Result_Prod.append(ciop.search(end_point = product_ref,params =[],output_fields='self,identifier,startdate,enclosure,startdate,enddate,wkt,title',creds='{}:{}'.format(cat_index,cat_apikey))[0] )
    

    input_metadata = gpd.GeoDataFrame.from_dict(Result_Prod)

    input_metadata['startdate'] = pd.to_datetime(input_metadata['startdate'])
    input_metadata['enddate'] = pd.to_datetime(input_metadata['enddate'])
    
    return input_metadata


def get_formatted_date(datetime_str):
    date = datetime.datetime.strftime(datetime_str, '%Y-%m-%dT%H:%M:%SZ')
 
    return date


def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    

    first_date_str = get_formatted_date(first_date)
    last_date_str = get_formatted_date(last_date)

    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date_str, last_date_str))
        file.write('geometry=%s' % (region_of_interest))
        
        
def get_anom_dates_from_circ (agg, LTA, first_year_agg, no_data_value):

    if agg == no_data_value or LTA == no_data_value:
        return no_data_value
    
    aug_1st = datetime.date(first_year_agg, 8, 1).timetuple().tm_yday # 1st aug
    ndays_year = datetime.date(first_year_agg, 12, 31).timetuple().tm_yday ## days of the first year

    if agg < aug_1st and LTA < aug_1st:
    
        anom = agg - LTA
    
    elif agg >= aug_1st and LTA >= aug_1st:
    
        anom = agg - LTA
    
    elif agg < aug_1st and LTA >= aug_1st:
    
        d1 = ndays_year - LTA
        d2 = agg
        anom = d1 + d2
    
    elif agg >= aug_1st and LTA < aug_1st:
    
        d1 = ndays_year - agg
        d2 = LTA
        anom = -(d1 + d2)
    
    return anom

def compute_anom_circ (agg_pds, lta_pds, first_year_agg, no_data_value):
    
    anoms_list = []
    index_list = []
    for (index_agg, value_agg), (index_lta, value_lta) in zip(agg_pds.items(), lta_pds.items()):
    
        index_list.append(index_agg)
        anoms_list.append(get_anom_dates_from_circ (value_agg, value_lta, first_year_agg, no_data_value))
    
    return pd.Series(anoms_list, index=index_list)

#### Workflow

Load data (agg)

In [ ]:
# organize indexes and apikeys in a python dictionary
indexAndApiKeys_splited = indexAndApiKeys['value'].split(',')
apikeys = {}
for idx,ele in enumerate(indexAndApiKeys_splited):
    
    if (idx % 2 == 0):
        print(ele)
        apikeys[ele] = indexAndApiKeys_splited[idx+1]
        

# get input data from catalog
input_metadata = get_input_metadata (input_references, apikeys)



input_metadata_LTA = input_metadata[input_metadata['title'].str.find('LTA') != -1]
input_metadata_Agg = input_metadata[input_metadata['title'].str.find('LTA') == -1]

#### Do start and end years belong to the same year

In [ ]:
start_year_agg = input_metadata_Agg['startdate'].iloc[0].year
end_year_agg = input_metadata_Agg['enddate'].iloc[0].year

same_year = True
if start_year_agg != end_year_agg:
    same_year = False
    
same_year

In [ ]:
# get file paths
filepath_agg = os.path.join(data_path, input_metadata_Agg['enclosure'].iloc[0].split('/')[-1])
filepath_LTA = os.path.join(data_path, input_metadata_LTA['enclosure'].iloc[0].split('/')[-1])

In [ ]:
var_names = ['start_growing_season', 'end_growing_season', 'smooth_ndvi', 'dif_ndvi', 'cumulative_ndvi', 'peak_ndvi']

In [ ]:
# load data into a python dictionary
# key -> variable name
# content -> list of pandas dataframe, one per season (TS)
data_agg = {}

f = filepath_agg
for var in var_names:

    df = pd.read_excel (f, sheet_name=var)
    
    # remove useless column
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
        
         
    if 'start_growing_season_date_avg' in df.columns:

        # drop date
        df = df.drop(columns=['start_growing_season_date_avg'])

        
    if 'start_growing_season_date_mode' in df.columns:

        # drop date
        df = df.drop(columns=['start_growing_season_date_mode'])

            
    if 'end_growing_season_date_avg' in df.columns:

        # drop date
        df = df.drop(columns=['end_growing_season_date_avg'])
        
    if 'end_growing_season_date_mode' in df.columns:

        # drop date
        df = df.drop(columns=['end_growing_season_date_mode'])
        
    
    data_agg[var] = df

Load data (LTA)

In [ ]:
# load data into a python dictionary
# key -> variable name
# content -> list of pandas dataframe, one per season (TS)
data_LTA = {}

f = filepath_LTA
for var in var_names:

    df = pd.read_excel (f, sheet_name=var)
    
    # remove useless column
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
        
    if 'start_growing_season_date_avg' in df.columns:

        # drop date
        df = df.drop(columns=['start_growing_season_date_avg'])

        
    if 'start_growing_season_date_mode' in df.columns:

        # drop date
        df = df.drop(columns=['start_growing_season_date_mode'])

            
    if 'end_growing_season_date_avg' in df.columns:

        # drop date
        df = df.drop(columns=['end_growing_season_date_avg'])
        
    if 'end_growing_season_date_mode' in df.columns:

        # drop date
        df = df.drop(columns=['end_growing_season_date_mode'])

    
    data_LTA[var] = df

Compute Amomalies

In [ ]:
# new python dictionary to store LTAs
anom_data = {}

anom_data = copy.deepcopy(data_agg)

# to each var computes mean
for var in var_names:
    
    cnames = data_agg[var].columns
    
    for c in cnames:
        
        if not('start_date' in c) and not('end_date' in c):
            
            if 'season' in c:
                
                if same_year:
                    anom_data[var][c] = data_agg[var][c] - data_LTA[var][c]
                else:
                    anom_data[var][c] = compute_anom_circ (data_agg[var][c], data_LTA[var][c], start_year_agg, -9999)
            else:
                anom_data[var][c] = data_agg[var][c].div(data_LTA[var][c])

#### write output

In [ ]:
name_parts = filepath_agg.split('/')[-1].split('.')[0].split('_')

mission = name_parts[0]
prod = name_parts[1]
aoi_name = name_parts[2]
start_date = name_parts[3]
end_date = name_parts[4]

name_parts = filepath_LTA.split('/')[-1].split('.')[0].split('_')

start_year_LTA = name_parts[4]
end_year_LTA = name_parts[5]

excel_output_name = '_'.join(['Anomaly', mission, prod, aoi_name, start_date, end_date, 'LTA', start_year_LTA, end_year_LTA]) + '.xlsx'
    
excel_output_name = os.path.join(output_folder, excel_output_name)

print(excel_output_name)


with pd.ExcelWriter(excel_output_name) as writer:  # doctest: +SKIP
    
    for key in var_names:
    
        anom_data[key].to_excel(writer, sheet_name=key)


#### write properties file

In [ ]:
write_properties_file(excel_output_name, input_metadata_Agg['startdate'].iloc[0], input_metadata_Agg['enddate'].iloc[0], regionOfInterest['value'])